In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import dlib
import requests


In [3]:

class GoogLeNetModified(models.GoogLeNet):
    def _transform_input(self, x):
        if x.size(1) == 1:
            # If the input has only one channel, replicate it to create three channels
            x = x.expand(-1, 3, -1, -1)

        x_ch0 = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
        x_ch1 = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
        x_ch2 = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
        x = torch.cat((x_ch0, x_ch1, x_ch2), 1)

        return x
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = GoogLeNetModified()
        # Modify the first layer to accept 3 channel input (for RGB images)
        self.features.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Modify the final layer to output the desired feature size
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)

        # Assuming the output is a tensor inside the GoogLeNetOutputs object
        logits_tensor = x.logits if hasattr(x, 'logits') else x  # Adjust accordingly based on the structure

        # Apply softmax directly on the logits tensor
        x_softmax = torch.nn.functional.softmax(logits_tensor, dim=1)
        return x_softmax






In [4]:
#import torch.nn as nn
#import torchvision.models as models
#from torchvision.models import googlenet

#class CNN(nn.Module):
 #   def __init__(self, num_classes):
  #      super(CNN, self).__init__()
  #      # Load the Inception GoogLeNet model
   #     self.features = models.inception_v3(pretrained=True)

   #     # Modify the input layer to accept 1 channel input (for grayscale spectrograms)
   #     self.features.Conv2d_1a_3x3.conv = nn.Conv2d(1, 32, kernel_size=3, stride=2, bias=False)

    #    # Modify the final fully connected layer to output the desired number of classes
    #    in_features = self.features.fc.in_features
    #    self.features.fc = nn.Linear(in_features, num_classes)

    #    self.softmax = nn.Softmax(dim=1)

    #def forward(self, x):
    #    x = self.features(x)
    #    x = self.softmax(x)
    #    return x


In [5]:
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229]),  # For grayscale images, only 1 channel
    ])
    img_tensor = transform(img)
    # If model expects RGB, convert grayscale images to RGB before feeding them:


    # img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension
    return img_tensor



In [6]:
def load_dataset(input_folder):
    X = []
    y = []
    # List all files in the input folder
    files = os.listdir(input_folder)
    # Iterate over files in the folder
    for filename in files:
        if filename.endswith(".png"):  # Assuming mel spectrograms are stored as PNG files
            input_path = os.path.join(input_folder, filename)
            img_tensor = preprocess_image(input_path)
            X.append(img_tensor)
            # Extract label from filename (assuming filename is in format "abc_IEO_label_xyz.png")
            label = filename.split("_")[2]
            if label == "HAP":
                y.append(0)
            elif label == "SAD":
                y.append(1)
            elif label == "ANG":
                y.append(2)
    return X, y

In [7]:
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [8]:
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [9]:
# def extract_features_from_folder(input_folder):
#     # Initialize the model
#     model = CNN(num_classes=3)  # 3 classes for HAPPY, SAD, ANGRY
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.eval()  # Set the model to evaluation mode

#     # List all files in the input folder
#     files = os.listdir(input_folder)

#     # Iterate over files in the folder
#     for filename in files:
#         if filename.endswith(".png"):  # Assuming mel spectrograms are stored as PNG files
#             input_path = os.path.join(input_folder, filename)
#             img_tensor = preprocess_image(input_path)
#             img_tensor = img_tensor.to(device)
#             with torch.no_grad():
#                 output_features = model(img_tensor)
#             print(f"Features extracted for {filename}: {output_features}")


In [10]:
# extract_features_from_folder('/content/drive/MyDrive/csci535/melspec')

In [11]:
# !python3 melspec_to_features_cnn.py /content/drive/MyDrive/csci535/melspec


In [12]:
if __name__ == "__main__":
    # Check if input arguments are provided
    # if len(sys.argv) != 2:
    #     print("Usage: python melspec_to_features_cnn.py input_folder")
    #     sys.exit(1)

    # input_folder = sys.argv[1]
    input_folder = '/content/drive/MyDrive/csci535/melspec'
    # Check if input folder exists
    if not os.path.exists(input_folder):
        print("Input folder does not exist.")
        sys.exit(1)

    # Load dataset and split into train and test sets
    X, y = load_dataset(input_folder)
    print(f"Total number of samples: {len(X)}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    print(f"Number of train samples: {len(X_train)}", f"Number of test samples: {len(X_test)}")
    # Initialize the model
    model = CNN(num_classes=3)  # 3 classes for HAPPY, SAD, ANGRY
    # Load the saved state dictionary
    # state_dict = torch.load('/content/drive/MyDrive/csci535/models/ResNet18_melspec_50_32_0.001')
    # state_dict = torch.load('/content/drive/MyDrive/csci535/models/ResNet18_video_50_32_0.001')
    # Load the state dictionary into the model
    # model.load_state_dict(state_dict)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Define loss function and optimizer
    _lr = 0.001
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=_lr)

    # Create data loaders
    _bs = 32
    train_loader = torch.utils.data.DataLoader(list(zip(X_train, y_train)), batch_size=_bs, shuffle=True)
    test_loader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), batch_size=_bs)
    print(f"Batch size: {_bs}", f"lr: {_lr}")
    # Training loop
    # num_epochs = 50
    # for epoch in range(num_epochs):
    #     print("Epoch " + str(epoch))
    #     train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
    #     test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
    #     print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    # Training loop
    #num_epochs = 50
    #for epoch in range(num_epochs):
     #   train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
      #  test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
       # print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Total number of samples: 273
Number of train samples: 191 Number of test samples: 82


/usr/local/lib/python3.10/dist-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Batch size: 32 lr: 0.001


100%|██████████| 3/3 [00:01<00:00,  1.87it/s]

Test Loss: 1.0993, Test Accuracy: 0.2805


In [13]:
# torch.save(model.state_dict(), 'ResNet18_melspec_'+str(num_epochs)+'_'+str(_bs)+'_'+str(_lr))

In [14]:
# ! ls -lh /content/

In [15]:
# !cp '/content/ResNet18_melspec_50_32_0.001' '/content/drive/MyDrive/csci535/models'